## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-31-07-40-59 +0000,nypost,Trump slams Pritzker on Chicago crime after at...,https://nypost.com/2025/08/31/us-news/trump-sl...
1,2025-08-31-07-24-58 +0000,wapo,Beaches across the U.S. close over fecal bacte...,https://www.washingtonpost.com/nation/2025/08/...
2,2025-08-31-07-24-07 +0000,bbc,First week 'critical' to avoid children missin...,https://www.bbc.com/news/articles/cg7jk3rr225o...
3,2025-08-31-07-18-58 +0000,bbc,Putin and Modi in China for summit overshadowe...,https://www.bbc.com/news/articles/clyrwv0egzro...
4,2025-08-31-05-33-48 +0000,nypost,Three victims of Florida 18-wheeler U-turn cra...,https://nypost.com/2025/08/31/us-news/three-vi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,22
6,killed,11
422,israeli,9
27,china,7
420,war,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
75,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...,58
0,2025-08-31-07-40-59 +0000,nypost,Trump slams Pritzker on Chicago crime after at...,https://nypost.com/2025/08/31/us-news/trump-sl...,48
70,2025-08-30-19-37-30 +0000,bbc,Trump administration moves to fire most Voice ...,https://www.bbc.com/news/articles/cdd38v3zvjdo...,46
120,2025-08-30-13-24-47 +0000,nyt,Trump Officials Move to Fire Most Voice of Ame...,https://www.nytimes.com/2025/08/29/us/politics...,44
48,2025-08-30-22-34-44 +0000,nyt,"Over 15 Killed in Gaza City, One Day After Isr...",https://www.nytimes.com/2025/08/30/world/middl...,43


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
75,58,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...
48,43,2025-08-30-22-34-44 +0000,nyt,"Over 15 Killed in Gaza City, One Day After Isr...",https://www.nytimes.com/2025/08/30/world/middl...
73,40,2025-08-30-19-00-34 +0000,nypost,Graphic video shows Israeli father throw himse...,https://nypost.com/2025/08/30/world-news/video...
52,28,2025-08-30-22-23-30 +0000,nypost,"NYC shooting vics were ‘targeted,’ police sour...",https://nypost.com/2025/08/30/us-news/nyc-shoo...
84,27,2025-08-30-17-25-31 +0000,nypost,Russia launches sweeping attack on Southern Uk...,https://nypost.com/2025/08/30/world-news/russi...
27,22,2025-08-31-01-07-38 +0000,startribune,"Saddened but hopeful, the Annunciation communi...",https://www.startribune.com/saddened-but-hopef...
70,22,2025-08-30-19-37-30 +0000,bbc,Trump administration moves to fire most Voice ...,https://www.bbc.com/news/articles/cdd38v3zvjdo...
88,22,2025-08-30-16-41-58 +0000,bbc,Prominent Ukrainian politician Andriy Parubiy ...,https://www.bbc.com/news/articles/cjw6ep37469o...
4,21,2025-08-31-05-33-48 +0000,nypost,Three victims of Florida 18-wheeler U-turn cra...,https://nypost.com/2025/08/31/us-news/three-vi...
86,20,2025-08-30-17-15-49 +0000,nyt,"Overlooked No More: Tina Modotti, Whose Life W...",https://www.nytimes.com/2025/08/30/obituaries/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
